In [ ]:
# Required imports
import os
import pandas as pd
import json

# First we extract the 'country_code', 'city_name', 'location_name' from the 'location' column and store these in new columns

In [ ]:
# Load the csv_file into a pandas dataframe
csv_file = 'csv_metadata.csv'
df = pd.read_csv(csv_file)

In [ ]:
# Function to extract the required fields
def extract_fields(location_entry):
    try:
        if isinstance(location_entry, str):  # Check if the entry is a string
            try:
                # Fix the internal JSON string first by converting the whole string into a Python dictionary
                location_dict = eval(location_entry)

               # Check if 'address_json' is not None before parsing
                if location_dict.get('address_json') is not None:
                    # Parse the JSON string in the 'address_json' field
                    address_info = json.loads(location_dict.get('address_json', '{}'))
                else:
                    address_info = {}  # Assign an empty dictionary if 'address_json' is None

                country_code = address_info.get('country_code', None)
                city_name = address_info.get('city_name', None)
                name = location_dict.get('name', None)

                return pd.Series([country_code, city_name, name])
            except (json.JSONDecodeError, SyntaxError) as e:
                print(f"Error decoding JSON: {location_entry}")
                print(f"Type: {type(location_entry)}")
                print(f"Exception: {e}")
                return pd.Series([None, None, None])
        else:
            # Return None for NaN or other unexpected types
            return pd.Series([None, None, None])
    except(TypeError) as e:
        print(f"Error decoding JSON: {location_entry}")
        print(f"Type: {type(location_entry)}")
        print(f"Exception: {e}")
        return pd.Series([None, None, None])

In [ ]:
# Extract the location information
df[['country_code', 'city_name', 'location_name']] = df['location'].progress_apply(extract_fields)

# Now we rename the column names for simplicity and drop unecessary columns

In [ ]:
# Defining a column name map for the renaming
column_mapping = {
    'owner.username': 'username',
    'owner.full_name': 'full_name',
    'owner.is_verified': 'is_verified',
    'owner.id': 'owner_id',
    'owner.is_private': 'is_private',
    'edge_media_preview_like.count': 'like_count',
    'count(edge_media_to_tagged_user.edges)': 'tagged_user_count',
    'id': 'post_id',
    'edge_media_to_parent_comment.count': 'comment_count',
    'taken_at_timestamp': 'timestamp',
    '__typename': 'post_type',
}

In [ ]:
# Renaming the columns according to the mapping
df.rename(columns=column_mapping, inplace=True)

In [ ]:
# Drop the 'location' column, as we have already extracted the relevant location data
modified_csv.drop(columns=['location', ], inplace=True)

In [ ]:
# Simply storing the final dataframe in the respective csv file
df.to_csv("metadata_with_extracted_location.csv")